## <span style="color: #0058bd;">Model Selection and Cross-Validation</span>


### <span style="color: #0058bd;">Exercise 35</span>
Four portfolios we have been looking at, and considering all 8 sets of
regressors which range from no factor to all 3 factors, which model is preferred
by AIC, BIC, GtS and StG?

In [ ]:
# Imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from itertools import product

In [ ]:
# Reading Fama-French dataset, exporting factors and portfolios (see 6_exercises_regression_basics.py for details and explanations)
ff = pd.read_hdf("../data/ff-pdr.h5", "ff")
factors = sm.add_constant(ff.iloc[:, :3])
portfolios = ff.iloc[:, 4:]
portfolios = portfolios[["SMALL LoBM", "SMALL HiBM", "BIG LoBM", "BIG HiBM"]]

#### <span style="color: #0058bd;">Our Approach: Criteria for Model Selection</span>

We will run multiple regression to determine the optimal model for explaining portfolio returns, using statistical criteria like the AIC (Akaike Information Criterion) and the BIC (Bayesian Information Criterion). 

**<span style="color: #0058bd;">Akaike Information Criterion:</span>** The criterion is derived from the concept of information entropy and wants to find the model that minimizes the loss of information. The formula is 
$$
    \text{AIC} = -2 \cdot \ln(\mathcal{L}) + 2k,
$$
where $\ln\left(\mathcal{L}\right)$ is the natural log of the likelihood of the model (indicating goodness of fit), and $k$ is the number of parameters in the model (penalty for complexity). A *lower AIC indicates a better model*. 

**<span style="color: #0058bd;">Bayesian Information Criterion:</span>** The criterion is derived from the concept of selecting the most probable model given the data (Bayesian framework). Compared to the AIC, the BIC *adds a more substantial penalty* for model complexity as the penalty for complexity grows with the logarithm of $N$. The formula is
$$
    \text{BIC} = -2 \cdot \ln(\mathcal{L}) + k \cdot \ln(N),
$$
where $N$ is the number of observations. Similar to the AIC, a *lower BIC suggests a better model*. 

**<span style="color: #0058bd;">A Philosophical Detour on the Differences:</span>** While both look similar, they are trying to answer two distinct questions. The AIC tries to select a model that is the closest description of an unknown, probably very high dimensional, truth (it assumes that all models are only approximations to a very high-dimensional truth). However, the truth is not in the set of candidate models that we consider. On the other hand, the BIC starts from a set of candidate models, and tries to find the true model among the set of candidate models. 

**<span style="color: #0058bd;">Practical Consideration:</span>** While not universally accepted, it is often a well-meant advice to report both AIC and BIC in your work. In many cases, they will agree on model selection. If they do not, specify the discrepancy across models. 

**<span style="color: #0058bd;">Application to our Context:</span>** In a linear regression model $Y = X\beta + \epsilon$ with $\epsilon_i \sim N(0, \sigma^2)$ both AIC and BIC use the likelihood function $\mathcal{L}(\beta, \sigma^2|Y)$. With normally distributed errors, the likelihood is related to the estimated variance of residuals $\hat{\sigma}^2$ and the RSS. One can show that normalized versions of AIC and BIC are
$$
    \text{AIC} = \ln \left(\hat{\sigma}^2\right) + 2 \cdot \frac{k}{N} ~~~~\text{ and }~~~~\text{BIC} = \ln \left(\hat{\sigma}^2 \right) + k \cdot \frac{\ln(N)}{N}.
$$ 
Smaller residual error means a better fit, and hence AIC and BIC will be lower.

In [ ]:
true_false = [True, False]
params = [true_false] * 3

In [ ]:
## We use itertools.product to get all possible combinations of True and False for the three factors
# product() function computes the Cartesian product of input iterables. It requires each iterable as a separate argument
# *params unpacks the list of lists into individual arguments for product()
choices = list(product(*params))
print(f"Number of different models: {len(choices)}")

In [ ]:
# We iterate over each portfolio return column and treat it as the dependent variable in the regression
for column in portfolios:
    # Initialize the dictionary to store the results
    results = {}

    for i in range(len(choices)):
        # Each regression includes a subset of factors (indicated by sel)
        sel = [True] + list(choices[i]) # always include the intercept
        x = factors.loc[:, sel]
        
        # Use list comprehension to initialize the portfolio names
        names = tuple(name for name in x.columns)
        res = sm.OLS(portfolios[column], x).fit()

        # Recover AIC and BIC from the regression results and store in results dictionary
        results[names] = [res.aic, res.bic]
    # Store the results in a DataFrame (for each combination of factors)
    ic = pd.DataFrame(results, index=["AIC", "BIC"]).T

    # Recover the model with the lowest AIC and BIC
    aic_model = ic.sort_values("AIC").index[0]
    bic_model = ic.sort_values("BIC").index[0]

    # For each portfolio, print the best model according to AIC and BIC
    print(f"For {column}, AIC selects {aic_model}")
    print(f"For {column}, BIC selects {bic_model}")

### <span style="color: #0058bd;">From General to Specific (GtS)</span>

For each of the portfolios, we start with a list of included variables that includes all three factors. We can then use a loop to see if any of the included variables have insignificant t-stats.  We first create a temporary set of regressors that uses the factors are are in `included`. We can then check if any of the t-stats are less than our critical value that is defined above.  If one is less than the value, we need to drop the variable. We sort the absolute t-stats so that the minimum is first, and then get the variable name from the index. Finally, we use `.remove` to remove this name from the list of included variables. 

If no t-stat is less than our critical value, we can call `break` which  terminates the loop early. 

In [ ]:
# Compute critical value from the normal distribution at 99.5% confidence level
cv = stats.norm.ppf(0.995); cv

In [ ]:
# Initialize a DataFrame to store the final portfolio-factor combinations
final_pf_factors = pd.DataFrame(columns=["Portfolio", "Included Variables (GtS)", "Included Variables (StG)"])

In [ ]:
final_pf_factors

In [ ]:
for column in portfolios: 
    included = list(factors.columns)
    y = portfolios[column]
    for i in range(3): 
        x = factors.loc[:, included]
        # Run OLS regression
        res = sm.OLS(y, x).fit(cov_type="HC0")
        # Recover t-values from the regressions
        tstats = res.tvalues
        # Test for significance of the lowest t-value
        if tstats.abs().min() < cv: 
            # If there is a t-value that is not significant, remove the minimum value
            sorted_tstats = tstats.abs().sort_values()
            remove = sorted_tstats.index[0]
            print(f"Portfolio {column} | Iteration {i+1}: Remove {remove} from the model")
            included.remove(remove)
        else:
            print(f"Portfolio {column} | Iteration {i+1}: Nothing to remove")
            break
    print(f"For {column}, GtS selects {included}")
    # Store the final set of factors for the given portfolio
    final_pf_factors = pd.concat(
        [final_pf_factors, pd.DataFrame({"Portfolio": [column], "Included Variables (GtS)": [included], "Included Variables (StG)": [[]]})], ignore_index=True
        )


In [ ]:
final_pf_factors

### <span style="color: #0058bd;">From Specific to General (StG)</span>

Instead of GtS, we can reverse the process and go from Specific to General (StG). For each of the portfolios, we start with an empty list of variables `included` and a list of excluded variables `excluded` that contains all three factors/regressors. 

We then successively add one of the excluded regressors, one at a time, and run an OLS regression with the current selection to obtain the t-stats. If the t-stat is larger than our critical value, we include the variable in `included`. We then continue the process and check whether we can add another of the factors in `excluded` to `included` by checking the t-stats. 

In [ ]:
for column in portfolios:
    # Store variables that pass the significance test
    included = []
    # Initially, all factors all excluded
    excluded = factors.columns
    # Portfolio returns for the current iteration
    y = portfolios[column]
    # Check which of the excluded factors can be added to the regression?
    for i in range(len(excluded)):
        tstats = {}
        # Iterate over all excluded variables
        for next_var in excluded:
            # Currently included and the next variable that we are considering to add
            col_names = included + [next_var]
            x = factors.loc[:, col_names]
            # Fit the regression
            res = sm.OLS(y, x).fit(cov_type="HC0")
            tstats[next_var] = res.tvalues.iloc[-1]
        tstats = pd.Series(tstats)
        # Check whether the new added variable is significant
        if tstats.abs().max() > cv:
            sorted_tstats = tstats.abs().sort_values()
            # Take the t-stat value from the candidate variable (the last one added)
            included = included + [sorted_tstats.index[-1]]
        else:
            break
        # Update the list of excluded variables (remove the last one added)
        excluded = set(factors.columns).difference(included)
    print(f"For {column}, StG selects {included}")
     # Update the GtS DataFrame with the included variables from StG procedure
    final_pf_factors.loc[final_pf_factors["Portfolio"] == column, "Included Variables (StG)"] = [included]


In [ ]:
final_pf_factors["Same Variables?"] = final_pf_factors.apply(
    # .apply() applies a function (or lambda function) to each row (axis = 1) of the DataFrame
    # For each row of the DataFrame, the lambda function is called
    # Each row of the DataFrame is passed as a Pandas Series to the lambda function
    lambda row: set(row["Included Variables (GtS)"]) == set(row["Included Variables (StG)"]), axis=1
)

In [ ]:
final_pf_factors

### <span style="color: #0058bd;">Exercise 36</span>

Cross-validation is a method of analyzing the **in-sample forecasting ability** of a
cross-sectional model by using $\alpha\%$ of the data to estimate the model and
then measuring the fit using the remaining $(100-\alpha)\%$. The most common forms
are 5- and 10-fold cross-validation which use $\alpha=20\%$ and $10\%$, respectively.
k-fold cross validation is implemented by randomly grouping the data into
k-equally-sized groups, using k-1 of the groups to estimate parameters, and
then evaluating using the bin that was held out. This is then repeated so that
each bin is held out.

1. Implement cross-validation using the 5- and 10-fold methods for all 8 models.
2. For each model, compute the full-sample sum of squared errors as well as the
   sum-of-squared errors using the held-out sample. Note that all data points
   will appear exactly once in both of these sum or squared errors. What happens
   to the cross-validated $R^{2}$ when computed on the held out sample when compared
   to the full-sample $R^{2}$? (k-fold cross validated SSE by the TSS).


##### <span style="color: #0058bd;">An Aside: Relationship Between MSE, SSE, and R-squared</span>

Remember that the Sum of Squared Errors (SSE) and the Total Sum of Squares (TTS) are given by 
$$
   \text{SSE} = \sum_{i=1}^N \left(y_i - \hat{y}_i\right)^2 ~~~~~\text{and}~~~~~\text{TSS} = \sum_{i=1}^N \left(y_i - \bar{y}\right)^2.
$$

Mean Squared Errors (MSE) are simply the normalized versions
$$
   \text{MSE}_{\text{resid}} = \frac{\text{SSE}}{N} ~~~~~\text{and}~~~~~\text{MSE}_{\text{total}} = \frac{\text{TSS}}{N}.
$$
Hence, $R^2$ in terms of MSE is given by
$$
   R^2 = 1 - \frac{\text{SSE}}{\text{TSS}}  = 1 - \frac{\text{MSE}_{\text{resid}}}{\text{MSE}_{\text{total}}}.
$$


In [ ]:
rs = np.random.RandomState(2309)

In [ ]:
# Initialize number of folds
folds = 10
# Retrieve number of observations
nobs = portfolios.shape[0]
# Re-shuffle all observations
order = list(rs.permutation(nobs))
block = nobs / folds

In [ ]:
for column in portfolios:
    portfolio = portfolios[column]
    model_errors = {}

    # Loop over different feature combinations
    for j in range(len(choices)):
        # We always want to include a constant
        sel = [True] + list(choices[j])
        model_factors = factors.loc[:, sel]
        # Same shape as our y-variable
        errors = portfolio.copy() 
        # START: Cross-validation part
        for i in range(folds):
            # Include all observations except for the i-th block (order was randomly shuffled)
            include = order[: int(i * block)] + order[int((i + 1) * block) :]
            # The i-th block is our hold-out sample
            hold_out = order[int(i * block) : int((i + 1) * block)]
            y = portfolio.iloc[include]
            x = model_factors.iloc[include]
            res = sm.OLS(y, x).fit()
            # Predict for the hold-out fold
            y_hat = res.predict(model_factors.iloc[hold_out])
            # Compute the prediction errors
            err = portfolio.iloc[hold_out] - y_hat
            errors.loc[err.index] = err
        model_name = tuple(factors.columns[sel])
        model_errors[model_name] = errors
    # Save model errors under the specific model name and convert from dictionary into DataFrame
    model_errors = pd.DataFrame(model_errors)
    # Compute the mean squared error (MSE) for each model and sort them
    mse = (model_errors**2).mean()
    mse = mse.sort_values()

    # Select the factors (index) of the best model according to the MSE and drop the NAs (if any)
    selected_factors = pd.Series(mse.index[0]).dropna()
    # Select the best model obtained through cross-validation
    x = factors[selected_factors]
    # Estimate the "best model" on the full dataset
    res = sm.OLS(portfolios[column], x).fit()
    print(f"\n For {column}, CV selects {tuple(selected_factors)}")
    print(
        f"The MSEs are {mse.iloc[0]} (CV) and {res.mse_resid} (full sample)"
    )
    # MSE_total will be the same for CV or full sample because "all data points will appear exactly once in both of squared errors"
    print(
        f"The R2s are {1 - mse.iloc[0] / res.mse_total} (CV) and {1 - res.mse_resid / res.mse_total} (full sample)"
    )

**Observation:** MSE is always larger in the CV than for the full sample. Similarly, $R^2$ is always lower in CV than in the full sample. Why? 

This is by construction. When using the full sample, the model is fitted using the entire dataset. The model is optimized to minimize residuals (SSE) for *all data points* because the same data is used for both training and testing. However, using CV, the model is evaluated on unseen data (test folds). More specifically, the model is trained on $k-1$ folds, but evaluated on the $k$-th fold, which was not part of the training. The model will not perform as well on the hold-out fold because it has never seen this data before. Hence, cross-validation penalizes overfitting because it tests the model on unseen data, where the model cannot explain noise or overfit to "seemingly irrelevant" details of the data. 

**Summary:** $\text{SSE}_{\text{CV}} \geq \text{SSE}_{\text{full}}$ and hence $R^2_{\text{CV}} \leq R^2_{\text{full}}$. 
